In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spectre.IO.Exporter import interpolate_to_points
from spectre.Visualization.OpenVolfiles import open_volfiles
from spectre.Visualization.ReadH5 import list_observations

In [ ]:
def load_data(var):
    dir = f"/Users/nilsvu/Downloads/scalar data/{dirname}"
    return np.loadtxt(dir + f"/{var}_{dirname}.dat")


def get_dirname(a, r, m):
    return f"a{a:.2f}_r{r:.2f}_m{m:d}"

In [ ]:
dirname = get_dirname(a=0.9, r=6, m=0)
rstars = load_data("rstars")
thetas = load_data("thetas")

print(rstars[0], rstars[-1])


def get_polarplot():
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={"projection": "polar"})
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_thetamin(0)
    ax.set_thetamax(180)
    return fig, ax


fig, ax = get_polarplot()
data = load_data("Seff")
ax.set_xlim(thetas[0], thetas[-1])
# ax.set_ylim(0., 5.)
contours = ax.contourf(
    data, extent=[thetas[0], thetas[-1], rstars[0], rstars[-1]], levels=1000
)
plt.colorbar(contours)

In [ ]:
h5files = [
    "/Users/nilsvu/Projects/spectre/build-Default-Debug/test_self_force_m0/ScalarSelfForceVolume0.h5"
]
obs_ids, _ = list_observations(open_volfiles(h5files, "/VolumeData"))
rr, tt = np.meshgrid(rstars, thetas)
data = (
    np.array(
        interpolate_to_points(
            h5files,
            "/VolumeData",
            observation_id=obs_ids[0],
            tensor_components=["FixedSource(MMode)_x"],
            # tensor_components=["BoyerLindquistRadius"],
            target_points=[rr.flatten(), tt.flatten()],
        )[0]
    )
    .reshape(rr.shape)
    .T
)

fig, ax = get_polarplot()
ax.set_xlim(thetas[0], thetas[-1])
# ax.set_ylim(2, 10)
contours = plt.contourf(
    np.log10(np.abs(data - load_data("Seff"))),
    extent=[thetas[0], thetas[-1], rstars[0], rstars[-1]],
    levels=1000,
)
plt.colorbar(contours)

In [ ]:
psi = (
    np.array(
        interpolate_to_points(
            h5files,
            "/VolumeData",
            observation_id=obs_ids[-1],
            tensor_components=["Re(SingularField)"],
            target_points=[rr.flatten(), tt.flatten()],
        )[0]
    )
    .reshape(rr.shape)
    .T
)

fig, ax = get_polarplot()
# ax.set_xlim(np.pi / 2 - np.pi / 6, np.pi / 2 + np.pi / 6)
ax.set_xlim(thetas[0], thetas[-1])
# ax.set_ylim(2, 10)
contours = plt.contourf(
    np.log10(np.abs(psi - load_data("psiS"))),
    # load_data("psiRe") + load_data("puncRe") - psi - data,
    extent=[thetas[0], thetas[-1], rstars[0], rstars[-1]],
    # levels=np.linspace(1, 2, 4),
    levels=1000,
)
plt.colorbar(contours)